#**6:**
**1. Entity-Relation Extraction**<br>
**2. Creation of Triples for KG along with properties of Nodes/Entities**<br>
`Input: process_flow_sentneces.csv, PMJAY_NLP_Model`<br>
`Output: triples.csv or triple.json`


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
pfs= pd.read_csv('/content/drive/MyDrive/Final (Completed)/process_flow_sentences_2.csv')

In [ ]:
pfs

,sentences,ent_1,lab_1,ent_2,lab_2
0,The Patient checks eligibility through Am I El...,Patient,PERSON,Am I Eligible App,MOBILE_APP
1,The Patient checks eligibility through Am I El...,Patient,PERSON,Am I Eligible Website,PORTAL
2,The Patient checks eligibility through Call-Ce...,Patient,PERSON,Call-Centre,ORG
3,The Patient checks eligibility through PMJAY K...,Patient,PERSON,PMJAY Kiosk,CSC
4,The Patient checks eligibility through EHCP Re...,Patient,PERSON,EHCP Registration Desk,CSC
...,...,...,...,...,...
97,General Specialist can also be MEDCO,General Specialist,PERSON,MEDCO,PERSON
98,Treating Specialist can also be MEDCO,Treating Specialist,PERSON,MEDCO,PERSON
99,Empanelled Hospital appoints a MEDCO,Empanelled Hospital,ORG,MEDCO,PERSON
100,PMJAY Beneficiary is a Patient,PMJAY Beneficiary,PERSON,Patient,PERSON


In [ ]:
sentences=pfs['sentences']
sentences

0      The Patient checks eligibility through Am I El...
1      The Patient checks eligibility through Am I El...
2      The Patient checks eligibility through Call-Ce...
3      The Patient checks eligibility through PMJAY K...
4      The Patient checks eligibility through EHCP Re...
                             ...                        
97                  General Specialist can also be MEDCO
98                 Treating Specialist can also be MEDCO
99                  Empanelled Hospital appoints a MEDCO
100                       PMJAY Beneficiary is a Patient
101                       Empanelled Hospital has Doctor
Name: sentences, Length: 102, dtype: object

## Extration of Entities

In [ ]:
import spacy
pmjay_nlp= spacy.load('/content/drive/MyDrive/Final (Completed)/nlp_model_2')

In [ ]:
#Extration of Entities from pfs sentences
#Create entity-pair dictionary and ent-label dictionary

ent_pair=[] # [(sentence,[ent_1, ent_2]),...]
el_dict={} #Entity-Label dictionary
for i in range(0,len(sentences)):
    e_pair=[]
    d= pmjay_nlp(sentences[i])
    for ent in d.ents:        
        e_pair.append(ent)
        el_dict[ent.text]=ent.label_
    if(len(e_pair)>1):
        ent_pair.append((sentences[i],e_pair))
print("Entity Pairs:",len(ent_pair),"\n",ent_pair)
print("Entity Label Dict:",len(el_dict),"\n",el_dict)

Entity Pairs: 94 
 [('The Patient checks eligibility through Call-Centre', [Patient, Call-Centre]), ('The Patient checks eligibility through PMJAY Kiosk', [Patient, PMJAY Kiosk]), ('The Patient checks eligibility through EHCP Registration Desk', [Patient, EHCP Registration Desk]), ('PMJAY Kiosk is assigned a PMAM', [PMJAY Kiosk, PMAM]), ('EHCP Registration Desk is assigned a PMAM', [EHCP Registration Desk, PMAM]), ('PMAM verifies identity and eligibility through BIS', [PMAM, BIS]), ('BIS creates Silver Record', [BIS, Silver Record]), ('BIS creates Golden Record', [BIS, Golden Record]), ('BIS creates e-card', [BIS, e-card]), ('e-card is given to the PMJAY Beneficiary', [e-card, PMJAY Beneficiary]), ('PMAM informs about entitlements to the PMJAY Beneficiary', [PMAM, PMJAY Beneficiary]), ('Patient becomes PMJAY Beneficiary', [Patient, PMJAY Beneficiary]), ('PMJAY Beneficiary is sent for Consultation', [PMJAY Beneficiary, Consultation]), ('Consultation is done by General Doctor', [Consulta

In [ ]:
#Adding Missing Data
el_dict['Am I Eligible App']='MOBILE_APP'
el_dict['Am I Eligible Website']='PORTAL'
el_dict['Medication']='PROCESS'
el_dict['Hospitalization']='PROCESS'
ent_pair.append((("The Patient checks eligibility through Am I Eligible App"),['Patient','Am I Eligible App']))
ent_pair.append((("The Patient checks eligibility through Am I Eligible Website"),['Patient','Am I Eligible Website']))
ent_pair.append((("General Doctor decides Hospitalization"),['General Doctor','Hospitalization']))
ent_pair.append((("General Specialist decides Hospitalization"),['General Specialist','Hospitalization']))
ent_pair.append((("General Doctor decides Medication"),['General Doctor','Medication']))
ent_pair.append((("General Specialist decides Medication"),['General Specialist','Medication']))
print("Entity Pairs:",len(ent_pair),"\n",ent_pair)
print("Entity Label Dict:",len(el_dict),"\n",el_dict)

Entity Pairs: 100 
 [('The Patient checks eligibility through Call-Centre', [Patient, Call-Centre]), ('The Patient checks eligibility through PMJAY Kiosk', [Patient, PMJAY Kiosk]), ('The Patient checks eligibility through EHCP Registration Desk', [Patient, EHCP Registration Desk]), ('PMJAY Kiosk is assigned a PMAM', [PMJAY Kiosk, PMAM]), ('EHCP Registration Desk is assigned a PMAM', [EHCP Registration Desk, PMAM]), ('PMAM verifies identity and eligibility through BIS', [PMAM, BIS]), ('BIS creates Silver Record', [BIS, Silver Record]), ('BIS creates Golden Record', [BIS, Golden Record]), ('BIS creates e-card', [BIS, e-card]), ('e-card is given to the PMJAY Beneficiary', [e-card, PMJAY Beneficiary]), ('PMAM informs about entitlements to the PMJAY Beneficiary', [PMAM, PMJAY Beneficiary]), ('Patient becomes PMJAY Beneficiary', [Patient, PMJAY Beneficiary]), ('PMJAY Beneficiary is sent for Consultation', [PMJAY Beneficiary, Consultation]), ('Consultation is done by General Doctor', [Consult

In [ ]:
#Extraction of Relations and creating Level 1 triplets for KG
import re
triplets_1=[] # [[ent_1, ent_2, relation],...]
for e in ent_pair:
    e1= str(e[1][0])
    e2= str(e[1][1])
    sent= e[0]
    result= re.search(e1+"(.*)"+e2,sent)
    triplets_1.append([e1,e2,result.group(1).strip()])
triplets_1 

[['Patient', 'Call-Centre', 'checks eligibility through'],
 ['Patient', 'PMJAY Kiosk', 'checks eligibility through'],
 ['Patient', 'EHCP Registration Desk', 'checks eligibility through'],
 ['PMJAY Kiosk', 'PMAM', 'is assigned a'],
 ['EHCP Registration Desk', 'PMAM', 'is assigned a'],
 ['PMAM', 'BIS', 'verifies identity and eligibility through'],
 ['BIS', 'Silver Record', 'creates'],
 ['BIS', 'Golden Record', 'creates'],
 ['BIS', 'e-card', 'creates'],
 ['e-card', 'PMJAY Beneficiary', 'is given to the'],
 ['PMAM', 'PMJAY Beneficiary', 'informs about entitlements to the'],
 ['Patient', 'PMJAY Beneficiary', 'becomes'],
 ['PMJAY Beneficiary', 'Consultation', 'is sent for'],
 ['Consultation', 'General Doctor', 'is done by'],
 ['Consultation', 'General Specialist', 'is done by'],
 ['PMJAY Beneficiary', 'General Doctor', 'consults a'],
 ['PMJAY Beneficiary', 'General Specialist', 'consults a'],
 ['General Doctor', 'Diagnostics', 'decides'],
 ['General Doctor', 'Pre-Authorization Form', 'fills'

In [ ]:
trip_df=pd.DataFrame(triplets_1)
trip_df.rename(columns={0:'source',1:'target',2:'edge'},inplace=True)
trip_df

,source,target,edge
0,Patient,Call-Centre,checks eligibility through
1,Patient,PMJAY Kiosk,checks eligibility through
2,Patient,EHCP Registration Desk,checks eligibility through
3,PMJAY Kiosk,PMAM,is assigned a
4,EHCP Registration Desk,PMAM,is assigned a
...,...,...,...
95,Patient,Am I Eligible Website,checks eligibility through
96,General Doctor,Hospitalization,decides
97,General Specialist,Hospitalization,decides
98,General Doctor,Medication,decides


In [ ]:
#trip_df.to_csv("/content/drive/MyDrive/Final (Completed)/triplets.csv",index=False)

In [ ]:
# Adding Properties to unique Entities
ue= pd.read_csv('/content/drive/MyDrive/Final (Completed)/unique_entities_added_properties.csv')
#ue['2']

In [ ]:
ue.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11
38,PMAM,"name:""PMAM""","full_form:""Pradhan Mantri Aarogya Mitra""","description:""They are trained professionals wh...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,SHA,"name:""SHA""","full_form:""State Health Authority""","description:""The State Health Agency is respon...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Empanelled Hospital,"name:""Empanelled Hospital""","description:""Empanelled hospitals are healthca...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TMS,"name:""TMS""","full_form:""Transaction Management System""","description:""It is an online platform develope...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,General Doctor,"name:""General Doctor""","description:""A doctor appointed to a PMJAY Ben...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Getting Properties for each Entity
ep_dict={} #dictionary containing entity:properties
ue_2=ue.drop(['0'], axis=1)
for i in range(0,len(ue_2)):
    d=(ue_2.iloc[i])
    prop=""
    for v in d:
        #print(v,":",type(v))
        if(type(v)==type('')):
            prop=prop+" "+str(v)+","
    prop=(prop[:-1]).strip()
    ep_dict[ue['0'][i]]=prop 
ep_dict

{'Treatment Procedure': 'name:"Treatment Procedure", description: "The standart treament guidelines can be downloaded from the website.", url: "https://www.pmjay.gov.in/resources/documents#stg"',
 'Insurance Company': 'name:"Insurance Company", description:"The colaborated Insurance Company will cover all the charges of the PMJAY Beneficiary subjected to their claim criteria."',
 'Medication': 'name:"Medication", description: "The medications are provided to every eligible PMJAY Beneficiary by the Empanelled Hospital or any other EHCP without any charges ."',
 'Am I Eligible App': 'name:"Am I Eligible App", description:"The Am I Eligible App is a moblie app using which a person can check his/her own eligibility under PMJAY Scheme."',
 'TMS': 'name:"TMS", full_form:"Transaction Management System", description:"It is an online platform developed by the National Health Authority (NHA) for managing the transactions and financial aspects of the scheme. TMS is used for various functions rela

In [ ]:
#assign unique variables to each entity 
#maintain a dict of {entity:variable} to use it for creating relations
def generate_variable(n):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    sequences = []
    if n < 26:
        return(alphabet[n])
    else:
        first = alphabet[(n // 26) - 1]
        second = alphabet[n % 26]
        return(first + second)
    return sequences
ev_dict={}
for i in range(0,len(ue)):
    var=generate_variable(i)
    e=str(ue['0'][i])
    ev_dict[e]=var 

In [ ]:
#Cypher Query for Creating unique nodes with properties
c_query=[]
for i in range(0,len(ue)):
    en=ue['0'][i]
    prop=ue['1'][i]
    q="CREATE ("+ev_dict[en]+":"+el_dict[en]+"{"+ep_dict[en]+"})"
    c_query.append(q)
c_query

['CREATE (a:DOCUMENT{name:"Treatment Procedure", description: "The standart treament guidelines can be downloaded from the website.", url: "https://www.pmjay.gov.in/resources/documents#stg"})',
 'CREATE (b:ORG{name:"Insurance Company", description:"The colaborated Insurance Company will cover all the charges of the PMJAY Beneficiary subjected to their claim criteria."})',
 'CREATE (c:PROCESS{name:"Medication", description: "The medications are provided to every eligible PMJAY Beneficiary by the Empanelled Hospital or any other EHCP without any charges ."})',
 'CREATE (d:MOBILE_APP{name:"Am I Eligible App", description:"The Am I Eligible App is a moblie app using which a person can check his/her own eligibility under PMJAY Scheme."})',
 'CREATE (e:PORTAL{name:"TMS", full_form:"Transaction Management System", description:"It is an online platform developed by the National Health Authority (NHA) for managing the transactions and financial aspects of the scheme. TMS is used for various fun

In [ ]:
#Use rel_pair to make relations cypher query
rel_pairs=[] #(source entity's variable, relation, target entitiy's variable)
for r in triplets_1:
    rel_pairs.append((ev_dict[r[0]],r[2],ev_dict[r[1]]))
rel_pairs 

[('z', 'checks eligibility through', 'ad'),
 ('z', 'checks eligibility through', 'al'),
 ('z', 'checks eligibility through', 'ai'),
 ('al', 'is assigned a', 'am'),
 ('ai', 'is assigned a', 'am'),
 ('am', 'verifies identity and eligibility through', 'w'),
 ('w', 'creates', 'ab'),
 ('w', 'creates', 'v'),
 ('w', 'creates', 'ah'),
 ('ah', 'is given to the', 't'),
 ('am', 'informs about entitlements to the', 't'),
 ('z', 'becomes', 't'),
 ('t', 'is sent for', 'x'),
 ('x', 'is done by', 'an'),
 ('x', 'is done by', 'ag'),
 ('t', 'consults a', 'an'),
 ('t', 'consults a', 'ag'),
 ('an', 'decides', 's'),
 ('an', 'fills', 'l'),
 ('l', 'is given to', 'aj'),
 ('aj', 'reports to', 'am'),
 ('ag', 'decides', 's'),
 ('ag', 'fills', 'l'),
 ('ac', 'provides', 'l'),
 ('am', 'sends', 'aa'),
 ('aa', 'is sent to', 'r'),
 ('aa', 'is sent to', 'o'),
 ('aa', 'is sent to', 'b'),
 ('r', 'approves', 'aa'),
 ('o', 'approves', 'aa'),
 ('b', 'approves', 'aa'),
 ('r', 'rejects', 'aa'),
 ('o', 'rejects', 'aa'),
 ('b', 

In [ ]:
#Cypher Query for Creating Relations between entities
# CREATE (a)-[:relation]->(b)
#import re
for r in rel_pairs:
    rel= re.sub(' ','_',r[1])
    c_query.append("CREATE ("+r[0]+")-[:"+rel+"]->("+r[2]+")")

for q in c_query:
    print(q)

CREATE (a:DOCUMENT{name:"Treatment Procedure", description: "The standart treament guidelines can be downloaded from the website.", url: "https://www.pmjay.gov.in/resources/documents#stg"})
CREATE (b:ORG{name:"Insurance Company", description:"The colaborated Insurance Company will cover all the charges of the PMJAY Beneficiary subjected to their claim criteria."})
CREATE (c:PROCESS{name:"Medication", description: "The medications are provided to every eligible PMJAY Beneficiary by the Empanelled Hospital or any other EHCP without any charges ."})
CREATE (d:MOBILE_APP{name:"Am I Eligible App", description:"The Am I Eligible App is a moblie app using which a person can check his/her own eligibility under PMJAY Scheme."})
CREATE (e:PORTAL{name:"TMS", full_form:"Transaction Management System", description:"It is an online platform developed by the National Health Authority (NHA) for managing the transactions and financial aspects of the scheme. TMS is used for various functions related to 

In [ ]:
'''for r in rel_pairs:
    rel= re.sub(' ','_',r[1])
    c_query.append("CREATE ("+r[0]+")-[:"+rel+"]->("+r[2]+")")
cypher_text=""
for q in c_query:
    cypher_text=cypher_text+q+"\n"
cypher_text=cypher_text.strip()'''

'for r in rel_pairs:\n    rel= re.sub(\' \',\'_\',r[1])\n    c_query.append("CREATE ("+r[0]+")-[:"+rel+"]->("+r[2]+")")\ncypher_text=""\nfor q in c_query:\n    cypher_text=cypher_text+q+"\n"\ncypher_text=cypher_text.strip()'

In [ ]:
cypher_text=""
for q in c_query:
    cypher_text=cypher_text+q+"\n"
cypher_text=cypher_text.strip()
print(cypher_text)

CREATE (a:DOCUMENT{name:"Treatment Procedure", description: "The standart treament guidelines can be downloaded from the website.", url: "https://www.pmjay.gov.in/resources/documents#stg"})
CREATE (b:ORG{name:"Insurance Company", description:"The colaborated Insurance Company will cover all the charges of the PMJAY Beneficiary subjected to their claim criteria."})
CREATE (c:PROCESS{name:"Medication", description: "The medications are provided to every eligible PMJAY Beneficiary by the Empanelled Hospital or any other EHCP without any charges ."})
CREATE (d:MOBILE_APP{name:"Am I Eligible App", description:"The Am I Eligible App is a moblie app using which a person can check his/her own eligibility under PMJAY Scheme."})
CREATE (e:PORTAL{name:"TMS", full_form:"Transaction Management System", description:"It is an online platform developed by the National Health Authority (NHA) for managing the transactions and financial aspects of the scheme. TMS is used for various functions related to 

In [ ]:
text_file = open("/content/drive/MyDrive/Final (Completed)/pmjay_process_flow.cypher", "w")
n = text_file.write(cypher_text)
text_file.close()

# End Of file